# Analyzing Playlists from Common Features

In [372]:
import requests
import pandas as pd
import numpy as np
import json
import constants

## Extraction of playlist data from Spotify's API

In [373]:
OAuth = constants.OAuth
headers = {"Accept" : "application/json",
           "Content-Type" : "application/json",
           "Authorization": OAuth}

In [374]:
def get_playlists(playlist_type, country_code, limit):
    
    '''
    Returns a dictionary of containing the given amount of playlists of the provided type from the 
    country matching the country code provided.
    '''
    
    playlists = (requests.get(url=f"https://api.spotify.com/v1/browse/categories/{playlist_type}/playlists?country={country_code}&limit={limit}",
                              headers = headers)).json()
    
    return playlists

In [375]:
def extract_playlist_ids(playlists):
    
    ''' Extract the ids of the playlists to be able to query the tracks contained in the playlist using Spotify API. '''
    
    ids = []
    playlist_info = playlists['playlists']['items']
    for i in range(len(playlist_info)):
        ids.append(playlist_info[i]['id'])
    
    return ids

In [376]:
def extract_track_info(playlist_ids):
    
    ''' Extract all of the relevant information about the tracks in a playlist, given a list a playlist ids. '''
    
    track_info = []

    for playlist in playlist_ids:
        tracks = (requests.get(url=f"https://api.spotify.com/v1/playlists/{playlist}/tracks",
                               headers = headers)).json()            
  
        fields = {"id", "name", "artists", "album", "duration_ms", "popularity"}
        for i in range(len(tracks['items'])):
            if tracks['items'][i]['track']:
                info = { key:value for key,value in tracks['items'][i]['track'].items() if key in fields}
                track_info.append(info)
            else:
                print('BARK BARK BARK >:(')
    
    return track_info

In [377]:
def get_tracks(playlist_type, country_code, limit):
    
    '''
    Returns a list containing the information for various tracks.
    playlist_type: The type of playlists where you want to query songs
    country_code: The country code of the country where you want to find the playlists
    limit: The amount of playlists you want to extract tracks from
    '''
    
    playlists = get_playlists(playlist_type, country_code, limit)
    playlist_ids = extract_playlist_ids(playlists)
    track_info = extract_track_info(playlist_ids)
    
    return track_info

In [378]:
def get_track_features(track_ids, df):
    '''
    Given an array of track ids extract the audio features of the track.
    '''
    track_features = []
    lower_bound = 0
    upper_bound = 60
    while lower_bound < len(track_ids):
        track_string = ','.join(df['id'][lower_bound:upper_bound])
        features = requests.get(url=f"https://api.spotify.com/v1/audio-features",
                                params={'ids':track_string},
                                headers=headers).json()
        lower_bound += 60
        upper_bound += 60
        if features['audio_features']:
            track_features.extend(features['audio_features'])
    return track_features

In [379]:
workout_tracks = pd.DataFrame(get_tracks('workout', 'US', 8))
workout_tracks = workout_tracks[['id', 'name', 'artists', 'album','duration_ms', 'popularity']]

display(workout_tracks.head())
print(workout_tracks.shape)

,id,name,artists,album,duration_ms,popularity
0,2KH16WveTQWT6KOG9Rg6e2,Eye of the Tiger,[{'external_urls': {'spotify': 'https://open.s...,"{'album_type': 'album', 'artists': [{'external...",245640,75
1,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,[{'external_urls': {'spotify': 'https://open.s...,"{'album_type': 'album', 'artists': [{'external...",208400,83
2,0pqnGHJpmpxLKifKRmU6WP,Believer,[{'external_urls': {'spotify': 'https://open.s...,"{'album_type': 'album', 'artists': [{'external...",204346,87
3,5BIMPccDwShpXq784RJlJp,Enter Sandman,[{'external_urls': {'spotify': 'https://open.s...,"{'album_type': 'album', 'artists': [{'external...",331573,73
4,57BrRMwf9LrcmuOsyGilwr,Crawling,[{'external_urls': {'spotify': 'https://open.s...,"{'album_type': 'album', 'artists': [{'external...",208960,73


(592, 6)


In [380]:
def extract_artist_name(track_data):
    artist_names = []
    for i in range(len(track_data)):
        artist_names.append(track_data['artists'][i][0]['name'])
    
    return artist_names

In [381]:
def extract_album_name(track_data):
    album_names = []
    for i in range(len(track_data)):
        album_names.append(track_data['album'][i]['name'])
    
    return album_names

In [382]:
workout_tracks['artists'] = extract_artist_name(workout_tracks)
workout_tracks['album'] = extract_album_name(workout_tracks)
workout_tracks.head()

,id,name,artists,album,duration_ms,popularity
0,2KH16WveTQWT6KOG9Rg6e2,Eye of the Tiger,Survivor,Rocky IV,245640,75
1,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,AC/DC,Highway to Hell,208400,83
2,0pqnGHJpmpxLKifKRmU6WP,Believer,Imagine Dragons,Evolve,204346,87
3,5BIMPccDwShpXq784RJlJp,Enter Sandman,Metallica,Metallica,331573,73
4,57BrRMwf9LrcmuOsyGilwr,Crawling,Linkin Park,Hybrid Theory (Bonus Edition),208960,73


## Extraction of Song Analyses

In [383]:
workout_track_features = pd.DataFrame(get_track_features(workout_tracks['id'], workout_tracks))
workout_track_features.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.13200,https://api.spotify.com/v1/audio-analysis/2KH1...,0.817,245640,0.599,2KH16WveTQWT6KOG9Rg6e2,0.000311,0,0.0873,-9.249,0,0.0328,108.873,4,https://api.spotify.com/v1/tracks/2KH16WveTQWT...,audio_features,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.548
1,0.05910,https://api.spotify.com/v1/audio-analysis/2zYz...,0.573,208400,0.913,2zYzyRzz6pRmhPzyfMEC8s,0.001730,6,0.1560,-4.793,0,0.1320,115.715,4,https://api.spotify.com/v1/tracks/2zYzyRzz6pRm...,audio_features,spotify:track:2zYzyRzz6pRmhPzyfMEC8s,0.422
2,0.06220,https://api.spotify.com/v1/audio-analysis/0pqn...,0.776,204347,0.780,0pqnGHJpmpxLKifKRmU6WP,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,https://api.spotify.com/v1/tracks/0pqnGHJpmpxL...,audio_features,spotify:track:0pqnGHJpmpxLKifKRmU6WP,0.666
3,0.00206,https://api.spotify.com/v1/audio-analysis/5BIM...,0.579,331573,0.824,5BIMPccDwShpXq784RJlJp,0.009030,6,0.0590,-8.710,0,0.0300,123.331,4,https://api.spotify.com/v1/tracks/5BIMPccDwShp...,audio_features,spotify:track:5BIMPccDwShpXq784RJlJp,0.635
4,0.04660,https://api.spotify.com/v1/audio-analysis/57Br...,0.580,208960,0.702,57BrRMwf9LrcmuOsyGilwr,0.000003,4,0.5360,-5.565,1,0.0337,105.076,4,https://api.spotify.com/v1/tracks/57BrRMwf9Lrc...,audio_features,spotify:track:57BrRMwf9LrcmuOsyGilwr,0.299


In [384]:
# Keeping only the important audio features
workout_track_features = workout_track_features[['id', 'acousticness', 'danceability','energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]
display(workout_track_features.head(5))
print(workout_track_features.shape)

,id,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2KH16WveTQWT6KOG9Rg6e2,0.13200,0.817,0.599,0.000311,0,0.0873,-9.249,0,0.0328,108.873,4,0.548
1,2zYzyRzz6pRmhPzyfMEC8s,0.05910,0.573,0.913,0.001730,6,0.1560,-4.793,0,0.1320,115.715,4,0.422
2,0pqnGHJpmpxLKifKRmU6WP,0.06220,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
3,5BIMPccDwShpXq784RJlJp,0.00206,0.579,0.824,0.009030,6,0.0590,-8.710,0,0.0300,123.331,4,0.635
4,57BrRMwf9LrcmuOsyGilwr,0.04660,0.580,0.702,0.000003,4,0.5360,-5.565,1,0.0337,105.076,4,0.299


(592, 13)


## Merging Data into a Consolidated Table

In [385]:
# Merge workout tracks so that all info is consolidated 
workout_tracks = pd.merge(workout_tracks, workout_track_features, on="id", how="inner").set_index('id')
workout_tracks.head()

,name,artists,album,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,,,,
2KH16WveTQWT6KOG9Rg6e2,Eye of the Tiger,Survivor,Rocky IV,245640,75,0.1320,0.817,0.599,0.000311,0,0.0873,-9.249,0,0.0328,108.873,4,0.548
2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,AC/DC,Highway to Hell,208400,83,0.0591,0.573,0.913,0.001730,6,0.1560,-4.793,0,0.1320,115.715,4,0.422
0pqnGHJpmpxLKifKRmU6WP,Believer,Imagine Dragons,Evolve,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
0pqnGHJpmpxLKifKRmU6WP,Believer,Imagine Dragons,Evolve,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
0pqnGHJpmpxLKifKRmU6WP,Believer,Imagine Dragons,Evolve,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666


In [386]:
print("Number of unique tracks:", workout_tracks.name.nunique())
print("Number of tracks:", len(workout_tracks))

Number of unique tracks: 545
Number of tracks: 640


### At this point we've extracted all of our necessary information. Note that there are repeated tracks, which makes sense since popular songs tend to be present in many different playlists.

In [387]:
# Out of curiosity, going to check the duplicate songs
workout_tracks.loc[workout_tracks.duplicated(subset='name')]['name'].unique()

array(['Believer', 'Another Day', 'Put Yo Hands Up', "'Till I Collapse",
       'Earthquake', "Ain't Scared (feat. Hashu)", 'Hood Anthem',
       'Lone Wolf', 'Something Just Like This', "If I Can't Have You",
       'Shape of You', 'Happier', '24K Magic', 'Rescue Me',
       'Who Do You Love', 'Never Really Over', 'The Middle',
       'More Than You Know', 'POWER', 'Stronger', 'Numb / Encore',
       "X Gon' Give It To Ya", 'Jumpman',
       'CAN\'T STOP THE FEELING! (Original Song from DreamWorks Animation\'s "TROLLS")',
       'ME! (feat. Brendon Urie of Panic! At The Disco)', 'Thunder',
       'Sucker', 'Cake By The Ocean', 'Stitches', 'Natural',
       "I Don't Care (with Justin Bieber)", 'You Need To Calm Down',
       'Sorry', 'Youngblood', '2002', 'Despacito - Remix', 'Attention',
       'Gold Digger', 'The Next Episode', 'Power', 'Say My Name'],
      dtype=object)

In [388]:
workout_tracks.drop_duplicates(inplace=True)

In [389]:
print("Number of unique tracks:", workout_tracks.name.nunique())
print("Number of tracks:", len(workout_tracks))

Number of unique tracks: 545
Number of tracks: 571


### After dropping duplicates there are still tracks with duplicate names. Before dropping them entirely, it'd be wise to investigate why they weren't dropped

In [390]:
workout_tracks[workout_tracks.duplicated(subset='name', keep=False)].sort_values(by='name').head(10)

,name,artists,album,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,,,,,
2BgEsaKNfHUdlh97KmvFyo,2002,Anne-Marie,Speak Your Mind (Deluxe),186986,83,0.0372,0.697,0.683,0.000000,1,0.1370,-2.881,0,0.1170,96.133,4,0.603
4Pbg79cTBu4vgSphoyNq3j,2002,Anne-Marie,2002,186986,77,0.0372,0.697,0.683,0.000000,1,0.1370,-2.881,0,0.1170,96.133,4,0.603
5cF0dROlMOK5uNZtivgu50,Attention,Charlie Puth,Voicenotes,208786,82,0.1090,0.775,0.613,0.000023,3,0.1340,-4.586,0,0.0542,100.066,4,0.797
4iLqG9SeJSnt0cSPICSjxv,Attention,Charlie Puth,Attention,211475,25,0.0969,0.774,0.626,0.000031,3,0.0848,-4.432,0,0.0432,100.041,4,0.777
0pqnGHJpmpxLKifKRmU6WP,Believer,Imagine Dragons,Evolve,204346,87,0.0622,0.776,0.780,0.000000,10,0.0810,-4.374,0,0.1280,124.949,4,0.666
05KfyCEE6otdlT1pp2VIjP,Believer,Imagine Dragons,Believer,203782,15,0.0417,0.772,0.775,0.000000,10,0.2260,-4.388,0,0.1120,124.978,4,0.748
6JV2JOEocMgcZxYSZelKcc,CAN'T STOP THE FEELING! (Original Song from Dr...,Justin Timberlake,CAN'T STOP THE FEELING! (Original Song from Dr...,236001,77,0.0123,0.667,0.830,0.000000,0,0.1910,-5.715,1,0.0749,113.030,4,0.701
1WkMMavIMc4JZ8cfMmxHkI,CAN'T STOP THE FEELING! (Original Song from Dr...,Justin Timberlake,TROLLS (Original Motion Picture Soundtrack),237546,74,0.0106,0.669,0.832,0.000000,0,0.0968,-5.720,1,0.0682,113.035,4,0.700
42ftjU4cTN5UTRksyqBKZJ,Cake By The Ocean,DNCE,SWAAY,218440,11,0.1470,0.771,0.760,0.000000,4,0.0530,-5.459,0,0.0514,119.000,4,0.881


### Reasons for Duplicates:
1. Same Song Different Artist
2. Same Song Same Artist Different Album
3. Same Everything, but Different Analysis  

#### Handle the second case and third case by dropping the less popular version.

In [391]:
workout_tracks = workout_tracks.sort_values(by=['name', 'popularity'], ascending = False)
workout_tracks.drop_duplicates(subset=['name', 'artists'], inplace=True)

In [392]:
# If this passes I can safely assume all duplicates have been removed
assert not any(workout_tracks.duplicated(subset=['name', 'artists'], keep=False))

### All duplicate records have been removed. Now we need to check data types and null values.

#### Checking for nulls

In [393]:
# If assertion error, then a column in the data contains a null value
assert not any(workout_tracks.isna().any())

#### Checking and adjusting dtypes

In [394]:
workout_tracks.dtypes

name                 object
artists              object
album                object
duration_ms           int64
popularity            int64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object

In [395]:
print(workout_tracks['time_signature'].value_counts())
print(workout_tracks['key'].value_counts())
print(workout_tracks['mode'].value_counts())

4    529
5     12
3      8
Name: time_signature, dtype: int64
1     102
2      57
11     50
0      49
7      44
5      44
6      42
8      38
9      36
10     35
4      32
3      20
Name: key, dtype: int64
1    322
0    227
Name: mode, dtype: int64


In [396]:
# The above 3 columns can be better represented as categories
workout_tracks[['time_signature', 'key', 'mode']] = workout_tracks[['time_signature', 'key', 'mode']].astype('category')

In [397]:
workout_tracks.dtypes

name                  object
artists               object
album                 object
duration_ms            int64
popularity             int64
acousticness         float64
danceability         float64
energy               float64
instrumentalness     float64
key                 category
liveness             float64
loudness             float64
mode                category
speechiness          float64
tempo                float64
time_signature      category
valence              float64
dtype: object

In [400]:
workout_tracks.describe()

,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000,549.000000
mean,209456.852459,57.856102,0.096731,0.677619,0.769250,0.054591,0.228989,-5.018740,0.161301,126.965124,0.468688
std,42070.842848,19.894413,0.142165,0.131779,0.147237,0.174861,0.189820,2.115448,0.132356,25.902920,0.223985
min,94245.000000,0.000000,0.000015,0.258000,0.291000,0.000000,0.028500,-15.290000,0.027500,70.023000,0.059100
25%,182192.000000,51.000000,0.008540,0.590000,0.668000,0.000000,0.099100,-6.076000,0.053100,106.970000,0.282000
50%,205792.000000,60.000000,0.035000,0.683000,0.785000,0.000000,0.149000,-4.822000,0.107000,127.988000,0.460000
75%,231826.000000,71.000000,0.132000,0.769000,0.895000,0.000557,0.316000,-3.638000,0.252000,148.050000,0.632000
max,369573.000000,100.000000,0.860000,0.967000,0.995000,0.971000,0.977000,0.101000,0.740000,191.971000,0.969000
